# Restore indexes

In [140]:
ES_HOST="http://localhost:9200"

## List if index exist

In [141]:
curl -s ${ES_HOST}/_cat/indices\?v

health status index uuid pri rep docs.count docs.deleted store.size pri.store.size


## Setting up variables

In [142]:
THEHIVE_INDEX="the_hive_15"
CORTEX_INDEX="cortex_4"

SNAPSHOT_PATH="the_hive_backup"
BACKUP_FOLDER="."


## Start with no index

 ⚠️  **DO NOT RUN THIS UNTIL YOU KNOW WHAT YOU ARE DOING** - **IF NOT GO TO** "Restore TheHive Index" **instead**

### Delete indices

In [132]:
echo -e "Run the following command to delete thehive index named $THEHIVE_INDEX:"
echo -e "curl -s XDELETE ${ES_HOST}/${THEHIVE_INDEX} | jq\n" 
echo -e "Run the following command to delete Cortex index named $CORTEX_INDEX"
echo -e "curl -s XDELETE ${ES_HOST}/${CORTEX_INDEX} | jq" 

Run the following command to delete thehive index named new_the_hive_15:
curl -s XDELETE http://localhost:9200/new_the_hive_15 | jq

Run the following command to delete Cortex index named new_cortex_4
curl -s XDELETE http://localhost:9200/new_cortex_4 | jq


### Delete TheHive Index

In [138]:
echo $NEW_THEHIVE_INDEX
curl -s -XDELETE ${ES_HOST}/$THEHIVE_INDEX | jq


{
  "acknowledged": true
}


### Delete Cortex Index

In [134]:
curl -s -XDELETE ${ES_HOST}/$CORTEX_INDEX | jq

{
  "acknowledged": true
}


## Restore TheHive Index

### Setup restore source folder

In [143]:
SNAPSHOT_PATH="the_hive_backup"


curl -s -XPUT ''${ES_HOST}'/_snapshot/'${SNAPSHOT_PATH}'' -H 'Content-Type: application/json' -d '{
    "type": "fs",
    "settings": {
        "location": "'${BACKUP_FOLDER}'",
        "compress": true
    }
}' | jq

{
  "acknowledged": true
}


### List backups

In [144]:
curl -s -XGET  -H' Content-Type: application/json' ''${ES_HOST}'/_cat/snapshots/'${SNAPSHOT_PATH}''

snapshot_1          SUCCESS 1517236911 14:41:51 1517236925 14:42:05 13.6s 17 41 0 41
snapshot_2          SUCCESS 1542307572 18:46:12 1542307573 18:46:13 849ms  1  5 0  5
snapshot_20181212   SUCCESS 1544617667 12:27:47 1544617667 12:27:47 545ms  1  5 0  5
before_demo         SUCCESS 1553003263 13:47:43 1553003265 13:47:45  2.2s  1  5 0  5
snapshot_3          SUCCESS 1554119454 11:50:54 1554119455 11:50:55  1.6s  1  5 0  5
snapshot_20200804_1 SUCCESS 1596548429 13:40:29 1596548429 13:40:29  22ms  0  0 0  0
snapshot_20200804_2 SUCCESS 1596550212 14:10:12 1596550215 14:10:15    3s  1  5 0  5
snapshot_20200304_3 SUCCESS 1599684050 20:40:50 1599684053 20:40:53  3.4s  1  5 0  5
snapshot_20200910_3 SUCCESS 1599684068 20:41:08 1599684068 20:41:08  84ms  1  5 0  5
snapshot_20200910_1 SUCCESS 1599684085 20:41:25 1599684085 20:41:25 117ms  1  5 0  5
thehive_20201027-1  SUCCESS 1603821541 17:59:01 1603821542 17:59:02  1.2s  1  5 0  5
thehive_20201027-2  SUCCESS 1603821692 18:01:32 1603821692 18:01:

### Restore TheHive data

In [145]:
THEHIVE_BACKUP_NAME="thehive_20201027-2"


curl -s -XPOST  -H 'Content-Type: application/json' \
''${ES_HOST}'/_snapshot/'${SNAPSHOT_PATH}'/'${THEHIVE_BACKUP_NAME}'/_restore' -d '{ 
"indices":"'"$THEHIVE_INDEX"'"
}' | jq

{
  "accepted": true
}


### List indexes

In [146]:
curl -s ${ES_HOST}/_cat/indices\?v

health status index       uuid                   pri rep docs.count docs.deleted store.size pri.store.size
yellow open   the_hive_15 KWvzTzq4RQOrSNCcaNstrw   5   0      26310           14        5mb            5mb


## Restore Cortex Index

### Setup restore source folder

In [147]:
BACKUP_FOLDER="."
CORTEX_SNAPSHOT_PATH="the_hive_backup"

curl -s -XPUT ''${ES_HOST}'/_snapshot/'${CORTEX_SNAPSHOT_PATH}'' -H 'Content-Type: application/json' -d '{
    "type": "fs",
    "settings": {
        "location": "'${BACKUP_FOLDER}'",
        "compress": true
    }
}' | jq

{
  "acknowledged": true
}


### List backups

In [148]:
curl -s -XGET  -H' Content-Type: application/json' ''${ES_HOST}'/_cat/snapshots/'${CORTEX_SNAPSHOT_PATH}''

snapshot_1          SUCCESS 1517236911 14:41:51 1517236925 14:42:05 13.6s 17 41 0 41
snapshot_2          SUCCESS 1542307572 18:46:12 1542307573 18:46:13 849ms  1  5 0  5
snapshot_20181212   SUCCESS 1544617667 12:27:47 1544617667 12:27:47 545ms  1  5 0  5
before_demo         SUCCESS 1553003263 13:47:43 1553003265 13:47:45  2.2s  1  5 0  5
snapshot_3          SUCCESS 1554119454 11:50:54 1554119455 11:50:55  1.6s  1  5 0  5
snapshot_20200804_1 SUCCESS 1596548429 13:40:29 1596548429 13:40:29  22ms  0  0 0  0
snapshot_20200804_2 SUCCESS 1596550212 14:10:12 1596550215 14:10:15    3s  1  5 0  5
snapshot_20200304_3 SUCCESS 1599684050 20:40:50 1599684053 20:40:53  3.4s  1  5 0  5
snapshot_20200910_3 SUCCESS 1599684068 20:41:08 1599684068 20:41:08  84ms  1  5 0  5
snapshot_20200910_1 SUCCESS 1599684085 20:41:25 1599684085 20:41:25 117ms  1  5 0  5
thehive_20201027-1  SUCCESS 1603821541 17:59:01 1603821542 17:59:02  1.2s  1  5 0  5
thehive_20201027-2  SUCCESS 1603821692 18:01:32 1603821692 18:01:

### Restore  data

In [149]:
CORTEX_BACKUP_NAME="cortex_20201027-2"

curl -s -XPOST  -H 'Content-Type: application/json' \
''${ES_HOST}'/_snapshot/'${CORTEX_SNAPSHOT_PATH}'/'${CORTEX_BACKUP_NAME}'/_restore' -d '{
"indices":"'${CORTEX_INDEX}'"
}' | jq

{
  "accepted": true
}


### List indexes

In [151]:
curl -s ${ES_HOST}/_cat/indices\?v

health status index       uuid                   pri rep docs.count docs.deleted store.size pri.store.size
green  open   the_hive_15 KWvzTzq4RQOrSNCcaNstrw   5   0      30998           16     33.2mb         33.2mb
green  open   cortex_4    Tu7saVsaRwWZ1X6VHdsg4w   5   0       8531            8      3.8mb          3.8mb
